# 3D Reconstruction

Imports:

In [1]:
import numpy as np
import cv2 as cv
import os
import camera_treatment
import feature_extraction
import ransacImplement
import triangulation
import matplotlib.pyplot as plt

Load dataset:

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"): # add file types as needed
            img = cv.imread(os.path.join(folder, filename))
            if img is not None:
                img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                images.append(img)
    return images

folder = "dataset"
images = load_images_from_folder(folder)

In [3]:
plt.imshow(images[0])

In [4]:
# Calibrate camera
K = np.array([[1518.317676, 0.000000, 582.870546],
    [0.000000, 1545.440926, 440.079523],
    [0.000000, 0.000000, 1.000000]])
print(K)

In [5]:
# Extract features
keypoints, descriptors = feature_extraction.multi_feature_extraction(images)
print(np.shape(keypoints))
print(np.shape(descriptors))

In [6]:
# Match features and keep only matching points
simple_matches, multi_matches = feature_extraction.multi_feature_matching(descriptors, window_size=20)

In [7]:
# Find Essential Matrices
fundamental_matrices = ransacImplement.simple_ransac_matrix(matches=simple_matches, keypoints=keypoints)
essential_matrices = ransacImplement.simple_get_essential_matrix(K, fundamental_matrices=fundamental_matrices)
print(np.shape(essential_matrices))

In [8]:
# Calculate projection matrices
proj_mats = [camera_treatment.ExtractCameraPose(E, K)[0] for E in essential_matrices]
Rs = [camera_treatment.ExtractCameraPose(E, K)[1] for E in essential_matrices]
Ts = [camera_treatment.ExtractCameraPose(E, K)[2] for E in essential_matrices]

print(np.shape(proj_mats))
proj_mats[0] = np.hstack((np.identity(3), np.transpose([np.zeros(3)])))

In [9]:
# Desambiguation
proj_mats_final = [proj_mats[0]]
for i in range(len(proj_mats) - 1):
    pts1 = np.array([keypoints[i][m.queryIdx].pt for m in simple_matches[i]]).T
    pts2 = np.array([keypoints[i+1][m.trainIdx].pt for m in simple_matches[i]]).T
    P2 = camera_treatment.disambiguate(pts1, pts2, proj_mats_final[i], proj_mats[i+1])
    proj_mats_final.append(P2)
print(np.shape(proj_mats_final))